# **EXAMEN PARCIAL**


In [1]:
#IMPORTACIÓN DE LIBRERÍAS

# En este apartado se realizará la importación de todas las librerías 
# a ser ocupadas para la creación del modelo como tersor flow , pandas ,sklearn 
# y numpy

import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import matplotlib.pyplot as plt


In [2]:
#CONEXIÓN A DRIVE Y LECTURA DEL DATASET

# El entorno de trabajo es colab por lo que es necesario importar el drive si nuestros archivos se 
# encuentran en este.

from google.colab import drive
drive.mount('/content/drive')

# Mediante la librería de pandas mandamos a buscar el archivo a la carpeta contenida en el drive
# una vez encontrado se colocaran tags a las columnas del dataframe

#El primer dataset se usa para la implementación del modelo de random forest, para la importancia de las características.
df1 = pd.read_excel("/content/drive/MyDrive/Colab/ResultadosCompleto.xlsx")

#Al primer dataset se le harán algunas modificaciones, por lo que se define un segundo dataset, con el mismo archivo. 
#Este segundo dataset se lo usa para la implementación del modelo
df = pd.read_excel("/content/drive/MyDrive/Colab/ResultadosCompleto.xlsx")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#INFORMACIÓN DEL DATASET
# Con el comando "info()", se obtiene el tipo de datos de las columnas y el número total de registros
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2210 entries, 0 to 2209
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   edad          2210 non-null   int64 
 1   genero        2210 non-null   object
 2   personalidad  2210 non-null   object
 3   estilo        2210 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 69.2+ KB


In [4]:
#VERIFICACIÓN DE VALORES NULOS EN EL DATASET

#Se debe comprobar que en el dataset no existan valores nulos, para que en los siguientes procesos, no se muestren resultados inconsistentes
check_for_nan = df1.isnull().values.any()
print (check_for_nan)



False


In [5]:
#TRANSFORMACIÓN DE CARACTERÍSTICAS CATEGÓRICAS A NUMÉRICAS PARA LA SELECCIÓN DE CARACTERÍSTICAS
#CON RANDOM FOREST

# Se realiza la respectiva transformación de los datos para que el modelo pueda entenderlos,
# como se puede ver en el siguiente algoritmo, todos los datos de las columnas de género y 
# personalidad se les asignará un número reemplazando los valores no numéricos

df1['genero'] = df1['genero'].map({'Masculino': 0,
                                 'Femenino': 1})

df1['personalidad'] = df1['personalidad'].map({'Confiable': 0,
                                           'Extravertido': 1,
                                           'Animado': 2,
                                           'Responsable': 3,
                                           'Serio': 4})

In [6]:
#DECLARACIÓN DE VARIABLES TEMPORALES PARA X E Y PARA LA SELECCIÓN DE CARACTERÍSTICAS

#Para las variables en X, seleccionamos todas las variables del dataset, excepto la variable que tendremos de salida: el estilo. 

X_inicial = df1.iloc[:,0:3]
y_inicial = df1.iloc[:,-1]
X_inicial

,edad,genero,personalidad
0,25,0,0
1,28,1,2
2,18,1,2
3,45,0,1
4,21,0,0
...,...,...,...
2205,26,1,2
2206,29,1,2
2207,22,0,1
2208,22,0,3


In [7]:
#El algoritmo de random forest de clasificación, contiene un comando que nos permite calcular el rango de importancia de cada variable.
#Así pues, según el resultado que se obtenga en este top, se comprueba si se está seleccionando correctamente las características. 

#En este paso, se realiza un entrenamiento parcial con las variables sin procesar. 
#Allí se consideran las variables independientes y la variable dependiente, tando de entrenamiento como de testeo. 
#Con este proceso, lo que se busca es listar a las características en orden ascendente, desde la más importante. 
#Debido a que solo se tienen 3 datos de entrada, el array consta de 3 variables. 


RF = RandomForestClassifier(max_depth=7, n_estimators=100)

df1 = pd.DataFrame()
RF.fit(X_inicial, y_inicial)
y_pred_RF= RF.predict(X_inicial)
cols = ['Machine Learning Classification Method',
        'Train Accuracy', 
        "Top 1 Feature Predictor",
        "Top 2 Feature Predictor",
        "Top 3 Feature Predictor"]

df1.loc['Machine Learning Classification Method', 
       'Details'] = 'Random Forest Classifier'
df1.loc['Train Accuracy', 'Details'] = RF.score(X_inicial, y_inicial) 


feature_importances = X_inicial.columns[np.argsort(RF.feature_importances_)][-3:]
df1.loc['Top 1 Feature Predictor', 'Details'] = feature_importances[2]
df1.loc['Top 2 Feature Predictor', 'Details'] = feature_importances[1]
df1.loc['Top 3 Feature Predictor', 'Details'] = feature_importances[0]

display(df1)

,Details
Machine Learning Classification Method,Random Forest Classifier
Train Accuracy,0.430769
Top 1 Feature Predictor,personalidad
Top 2 Feature Predictor,edad
Top 3 Feature Predictor,genero


**AHORA QUE SE TIENE UNA LISTA DE IMPORTANCIA DE LAS CARACTERÍSTICAS SE PROCEDE A LAS DEBIDAS TRANSFORMACIONES**

In [8]:
#TRANSFORMACIÓN DE EDAD CON LA TÉCNICA DE BINNING

#Se coloca el intervalo de edades que está permitido en el dataset (de 17 a 30 años, según el objetivo de análisis del proyecto)
#donde se dividirán en 5 rangos, que estarán etiquetados con los nombres de la variable "labels"
bins = np.linspace(17,30,6)
labels = ['0.1', '0.2', '0.3','0.4','0.5']
df['edad'] = pd.cut(df['edad'], bins, labels=labels)

print(bins)
print(df)

[17.  19.6 22.2 24.8 27.4 30. ]
     edad     genero  personalidad  estilo
0     0.4  Masculino     Confiable       5
1     0.5   Femenino       Animado       4
2     0.1   Femenino       Animado       6
3     NaN  Masculino  Extravertido       3
4     0.2  Masculino     Confiable       3
...   ...        ...           ...     ...
2205  0.4   Femenino       Animado      10
2206  0.5   Femenino       Animado       1
2207  0.2  Masculino  Extravertido       8
2208  0.2  Masculino   Responsable       4
2209  0.1  Masculino     Confiable       7

[2210 rows x 4 columns]


In [9]:
#VERIFICACIÓN DE LOS TIPOS DE DATOS LUEGO DE LA APLICACIÓN DE TÉCNICA DE BINNING

#Al utilizar binning para colocar los datos en rangos, se puede observar que el tipo de dato se cambió a categórico
#Esto indica que más adelante, se deberá realizar nuevamente una transformación para que se vuelva un dato de tipo numérico.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2210 entries, 0 to 2209
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   edad          1956 non-null   category
 1   genero        2210 non-null   object  
 2   personalidad  2210 non-null   object  
 3   estilo        2210 non-null   int64   
dtypes: category(1), int64(1), object(2)
memory usage: 54.3+ KB


In [10]:
#VERIFICACIÓN DE DATOS NULOS 

#Es importante que luego de hacer una modificación de alguna variable en el dataset, se vuelva a comprobar que no existan datos nulos.
check_for_nan = df.isnull().values.any()
print (check_for_nan)

#Con el siguiente comando, se imprimen la cantidad de nulos que existen y en qué columnas se encuentran. 
#Solo en la edad existen datos nulos, ya que existían personas que no tenían la edad dentro del rango permitido; por tanto, no se les otorgó un
#intervalo con el proceso de binning y pasaron a ser datos nulos.
df.isnull().sum()

True


edad            254
genero            0
personalidad      0
estilo            0
dtype: int64

In [11]:
#COMO SI EXISTEN VALORES NULOS, SE LOS DEBE ELIMINAR

#Como los datos nulos, son de las edades que no corresponden al intervalo escogido para la muestra, se deben eliminar. 
#De esta manera, de los 2209 registros que se tenía en un inicio, se disminuyeron a 1956.
df_nulos = df[~df['edad'].isnull()].copy()
df_nulos

,edad,genero,personalidad,estilo
0,0.4,Masculino,Confiable,5
1,0.5,Femenino,Animado,4
2,0.1,Femenino,Animado,6
4,0.2,Masculino,Confiable,3
5,0.2,Masculino,Animado,10
...,...,...,...,...
2205,0.4,Femenino,Animado,10
2206,0.5,Femenino,Animado,1
2207,0.2,Masculino,Extravertido,8
2208,0.2,Masculino,Responsable,4


In [12]:
#TÉCNICA ONE_HOT_ENCONDE PARA LA TRANSFORMACIÓN DE VARIABLES CATEGÓRICAS A NUMÉRICAS
one_hot_genero = pd.get_dummies(df.genero ,prefix='genero')
one_hot_Personalidad = pd.get_dummies(df.personalidad ,prefix='personalidad')

#AL IMPLEMENTAR EL BINNING EN LA EDAD, ESTOS DATOS SE TRANSFORMAN EN CATEGÓRICOS. POR LO TANTO
#SE APLICA ONE_HOT_ENCODE TAMBIÉN A LA EDAD
one_hot_Edad = pd.get_dummies(df.edad ,prefix='edad')

#CONCATENACIÓN DE LAS VARIABLES TRANSFORMADAS
d = pd.concat([one_hot_Edad,one_hot_genero, one_hot_Personalidad,df.estilo], axis=1)
d

,edad_0.1,edad_0.2,edad_0.3,edad_0.4,edad_0.5,genero_Femenino,genero_Masculino,personalidad_Animado,personalidad_Confiable,personalidad_Extravertido,personalidad_Responsable,personalidad_Serio,estilo
0,0,0,0,1,0,0,1,0,1,0,0,0,5
1,0,0,0,0,1,1,0,1,0,0,0,0,4
2,1,0,0,0,0,1,0,1,0,0,0,0,6
3,0,0,0,0,0,0,1,0,0,1,0,0,3
4,0,1,0,0,0,0,1,0,1,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2205,0,0,0,1,0,1,0,1,0,0,0,0,10
2206,0,0,0,0,1,1,0,1,0,0,0,0,1
2207,0,1,0,0,0,0,1,0,0,1,0,0,8
2208,0,1,0,0,0,0,1,0,0,0,1,0,4


In [13]:
#VERIFICACIÓN DE LOS TIPOS DE DATOS QUE SE TRANSFORMARON
#Luego de realizar todas las transformaciones, obtenemos los tipos de datos que se encuentran en el dataset, para confirmar
#que ya no se necesiten más transformaciones
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2210 entries, 0 to 2209
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   edad_0.1                   2210 non-null   uint8
 1   edad_0.2                   2210 non-null   uint8
 2   edad_0.3                   2210 non-null   uint8
 3   edad_0.4                   2210 non-null   uint8
 4   edad_0.5                   2210 non-null   uint8
 5   genero_Femenino            2210 non-null   uint8
 6   genero_Masculino           2210 non-null   uint8
 7   personalidad_Animado       2210 non-null   uint8
 8   personalidad_Confiable     2210 non-null   uint8
 9   personalidad_Extravertido  2210 non-null   uint8
 10  personalidad_Responsable   2210 non-null   uint8
 11  personalidad_Serio         2210 non-null   uint8
 12  estilo                     2210 non-null   int64
dtypes: int64(1), uint8(12)
memory usage: 43.3 KB


In [14]:
#NORMALIZACIÓN DE LA VARIABLE DE SALIDA (ESTILO)

#Como la variable de salida, no se encuentra en el mismo rango que el resto de los datos, se procede a realizar la normalización de esta variable.
#Para ello, se define una variable que contenga nuestros datos de salida
y_for_norm = df.iloc[:,-1]

#Y se realiza la normalización de Mínimo y máximo, para otorgarle valores al estilo de entre 0 y 1.
def minmax_norm(variable):
    return (variable - variable.min()) / ( variable.max() - variable.min())

y_norm = minmax_norm(y_for_norm)

print(y_norm)

0       0.444444
1       0.333333
2       0.555556
3       0.222222
4       0.222222
          ...   
2205    1.000000
2206    0.000000
2207    0.777778
2208    0.333333
2209    0.666667
Name: estilo, Length: 2210, dtype: float64


In [15]:
#CONCATENACIÓN DE TODAS LAS VARIABLES TRANSFORMADAS

#Ahora que ya se encuentran todas las variables transformadas, se procede a unirlas en una sola tabla.
d = pd.concat([one_hot_Edad,one_hot_genero, one_hot_Personalidad,y_norm], axis=1)
d

,edad_0.1,edad_0.2,edad_0.3,edad_0.4,edad_0.5,genero_Femenino,genero_Masculino,personalidad_Animado,personalidad_Confiable,personalidad_Extravertido,personalidad_Responsable,personalidad_Serio,estilo
0,0,0,0,1,0,0,1,0,1,0,0,0,0.444444
1,0,0,0,0,1,1,0,1,0,0,0,0,0.333333
2,1,0,0,0,0,1,0,1,0,0,0,0,0.555556
3,0,0,0,0,0,0,1,0,0,1,0,0,0.222222
4,0,1,0,0,0,0,1,0,1,0,0,0,0.222222
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2205,0,0,0,1,0,1,0,1,0,0,0,0,1.000000
2206,0,0,0,0,1,1,0,1,0,0,0,0,0.000000
2207,0,1,0,0,0,0,1,0,0,1,0,0,0.777778
2208,0,1,0,0,0,0,1,0,0,0,1,0,0.333333


In [19]:
#Definidos ya todos los valores, se exporta el dataset con el que se trabajará en los siguientes procesos. 
d.to_excel("[NRC_8888]VictoriaMendoza_EvaluaciónEnLineaUnidad3.xlsx")

#En new_data se almacena el dataset que se acaba de exportar, para que, mediante esta variable, se llame al nuevo dataframe
new_data = pd.read_excel("[NRC_8888]VictoriaMendoza_EvaluaciónEnLineaUnidad3.xlsx")

In [20]:
#DEFINICIÓN DE VARIABLES DE ENTRADA Y LA VARIABLE DE SALIDA

#Ya teniendo todos los datos transformados, se puede definir "x" e "y" para trabajar con ellas, en el proceso posterior e implementarlas ya en el modelo
X = new_data.iloc[:,0:12]
y = new_data.iloc[:,-1].values

In [21]:
#DIVISIÓN DEL DATASET PARA LOS DATOS DE ENTRENAMIENTO Y LOS DE TESTEO

#Finalmente, se dividen los datos para que sean 70% para el entrenamiento y el 30% para el testeo.
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y, 
                                                    test_size=0.3, random_state=100)

## **INTERPRETACIÓN DE RESULTADOS** 

El dataset utilizado cuenta con 2010 datos, los cuales se obtuvieron a través de una encuesta, donde todos los campos eran obligatorios. Existen solo 3 variables de entrada y 1 de salida. Antes de realizar cualquier proceso, se obtuvo una lista de las características más importantes del dataset, con Random Forest, para tener claro cuáles debían ser utilizadas. Por ello, solo se transformó las dos variables categóricas a numéricas, de forma genérica con el comando "map". Ese dataframe transformado se pasó al modelo de random forest y así se obtuvo que la primera variable importante a utilizar, es la personalidad. 

Luego de obtener la importancia de las características, se utilza el dataset sin transformar. Allí se separa primero la edad, para aplicar una técnica de binning adapatativo y tener la edad en rangos de 0 a 1. Debido a que la edad, debe estar en un intervalo de 17 a 30 años, según el objetivo de estudio de nuestro modelo, las edades que no constan en ese rango, luego de aplicar Binning, aparecerán como datos nulos. Por ello, se verifica la cantidad de datos nulos y se los elimina. 
Las variables categóricas también deben ser transformadas a valores numéricos. Para ello se utiliza la técnica de One Hot Encoding. Al hacer un print de los tipos de valores, se observa que la edad pasó a ser un dato categórico, por lo que también se le aplica One Hot Encoding y se transforman las variables. 
En este paso, se deberían cambiar los tipos de datos a enteros para no tener problemas luego con el modelo. Debido al tiempo, este proceso se lo realizará luego y se volverá a formar el archivo plano. 

La personalidad también es una variable que se debe tratar, asi que se aplica la técnica de normalización, para que todo el dataset contenga datos dentro del mismo intervalo. 
Finalmente, luego de generar el archivo plano, se declaran las X y la Y de nuestro modelo, y se divide el dataset entre los datos de entrenamiento y los de testeo. 

### **Qué modelo de ML/DL/DM usted aplicará a su proyecto para ser resuelto?**
  Según los artículos analizados, referentes al desarrollo de sistemas para la predicción de moda y personalidad, se utiliza mayormente redes neuronales y Support Vector Machine. Por esta razón, vamos a implementar estos mismos modelos. Se utiliza redes neuronales simples, porque el problema no tiene una complejidad tan grande y el conjunto de datos es relativamente pequeño, para trabajar con estos modelos. Por otro lado, SVM presenta la ventaja que es utilizada precisamente para grupos de datos pequeños. Además, que se utilizarán 3 kernels para la comparación de los resultados.



### **Con qué modelos comparará ese modelo y por qué?**
Luego de implementar Redes Neuronales Simples, se usará también SVM para comparar la precisión del primer modelo y del segundo. En SVM se implementan 3 Kernels: polinomial, Radial Basis Function (RBF) y sigmoidal. 
Como se mencionó en la pregunta anterior, se usa estos modelos porque son los más recurrentes en este tema de predicciones. Además, SVM nos pareció muy conveniente, ya que nuestro dataset solo contiene 2010 datos y este modelo, puede obtener buenos resultados con datasets pequeños. 

### **Cuáles son los hyperparameters que utilizará para cada modelo? Interprete los resultados**

Los hyperparameters que se utilizará para redes neuronales son: input_shape, epoch, alpha, optimizer, loss y metrics. Con estos parámetros se pretende tener una mejora en los resultados de predicción. Por su parte, el input_shape contiene el numero de entradas de la red. Epoch, define el número de veces que se envían los datos al modelo. Alpha es el porcentaje de aprendizaje de los datos. Para el optimizador se tienen algunas opciones, pero en este ejemplo, se usará la técnica de "Adam". Loss representa el porcentaje pérdido del aprendizaje y metrics, evalúa el resultado con una salida de "acurracy" del modelo. 

Para SVM, se usan "Gamma" y "regularización". Este segundo parámetro, es representado con la letra "C" y, como su nombre lo indica, trata de regularizar el porcentaje de error que se tiene. La regularización, envía un mensaje al modelo, para avisarle cuánto porcentaje de error, puede soportar. Por otro lado, Gamma define la distancia influyente en el cálculo de la línea de separación aceptable. Con el hiperplano generado, gamma debe calcular la separación indicada de este, con los puntos. Con este parámetro, se debe tener mucho cudiado ya que puede afectar severamente los resultados de predicción. 
Ambos parámetros son los más utilizados en SVM. 


### **Qué métricas utilizará para evaluar su modelo? Comente sobre cada métrica y qué valor espera.**

El acurracy es la primera métrica que se pretende analizar, ya que con ella tendremos los resultados donde el modelo predijo correctamente el estilo de vestir de cada persona. La entrada principal del dataset es la personalidad. De ella existen 5 categorías, mientras que la salida, que son los estilos de vestir, son 10. 
Con el acurracy mediremos el porcentaje donde el modelo enlazó cada personalidad con un estilo, desde la perspectiva de la predicción. Es decir, se evalúa solo lo correcto que ha predecido. Aquí es espera tener al menos un 75% u 80% de acertación. Quizás estos valores no son muy altos, pero sería un gran acercamiento, ya que no encontramos otro artículo donde se desarrolle un sistema que permita la predicción de estilos de vestir, en base a la personalidad.


Por otro lado, se evaluará también la precisión ya que con esta métrica podemos obtener un resultado de los estilos predichos correctamente y de los reales positivos. Es decir, con la precisión evaluaremos tanto la parte de predicción desde esa perspectiva, así como desde la perspectiva de datos reales. 
Si en un caso, la personalidad confiable está enlazada con un estilo "formal", entonces se evalúa en que casos el modelo predijo que era formal y efectivamente lo era. 
